# Quick Start Guide

Import the *toolkit* from the *zhinst* namespace as follows:


In [1]:
import zhinst.toolkit as tk

## Initialize the device

The *toolkit* provides instrument drivers for multiple different *Zurich Instruments* devices. It includes drivers for

* HDAWG
* UHFQA
* PQSC
* UHFLI
* MFLI

All of the instrument drivers available in the *toolkit* share some basic device properties. Every device is identified by a `name` and a `serial` number. The `name` is free for the user to chose and only for their convenience. The `serial` however, is a unique identifier for every *Zurich Instruments* device. The serial number can be found on the back panel of instrument and is of the format `DEV1234`. In addition, the *toolkit* needs to know what `interface` the device uses and where to find the data server (`host`). The value for the interface defaults to a connection via Ethernet (`1GbE`) and the host address is `localhost` unless specified otherwise.

As a first example, we now initialize the instrument driver for a HDAWG. We name it `hdawg1` and we know that our device's serial number is `dev8030`.

In [2]:
hdawg = tk.HDAWG("hdawg1", "dev8345", interface="usb")

hdawg.setup()           # set up data server connection
hdawg.connect_device()  # connect device to data server

Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV8345 on interface USB


The methods `setup()` and `connect_device()` initialize the data server and connect the device to it. Now the device is connected and ready to go.

Every instrument driver in the *toolkit* features some properties with basic information about the device. What was the device's serial number again? What options are installed?

In [3]:
print(f"name:        {hdawg.name}")
print(f"serial:      {hdawg.serial}")
print(f"device:      {hdawg.device_type}")
print(f"options:     {hdawg.options}")
print(f"interface:   {hdawg.interface}")
print(f"connected:   {hdawg.is_connected}")

name:        hdawg1
serial:      dev8345
device:      DeviceTypes.HDAWG
options:     ['']
interface:   usb
connected:   True


## Access the device's nodetree  

The driver's property `nodetree` is a data structure that allows the user to access all the settings and data on the device. The settings are highly structured into logical groups, enumerations and options. 

For example, all of the device's signal inputs and all of its signal outputs are grouped together. Also the HDAWG's eight sine generators are grouped into `sines` that are then enumerated from `0 - 7`. Each `Node` in the nodetree as well as the nodetree itself can have other `Nodes` or `Parameters` as attributes. The `Parameters` are the 'leaves' of the tree data structure and represent individual settings or data on the instrument.

```
HDAWG
└── nodetree   
    │
    └─── sigouts             <--- Node List
    │    └─── 0              <--- Node
    │    │    └─── on        <--- Prameter
    │    │    └─── range 
    │    │    └─── direct
    │    │    └─── offset
    │    │    └─── ...
    │    └─── 1  
    │    └─── ...  
    │
    └─── oscs 
    │    └─── 0
    │    │    └─── freq
    │    └─── 1
    │    └─── 2
    │    └─── ...
    │
    └─── sines
    │    └─── ...
    │
    └─── dio
    │    └─── ...
    │
    └─── ...     
```

Enumerated nodes such as the eight sine-generators or 16 oscillators of the HDAWG are grouped together in lists. For example the node of the first sine-generator would be accessed via `hdawg.nodetree.sines[0]`. 

In a *Jupyter* notebook or a console it is easy to navigate through the `NodeTree`. At every given node, the object representation shows a list of all the available sub-nodes and parameters. In this way it is easy to find the exact setting you might need.

In [4]:
hdawg.nodetree

nodes:
 - stats
 - oscs
 - status
 - sines
 - dio
 - awgs
 - system
 - sigouts
 - triggers
 - features
parameters:
 - clockbase

At the end of the tree, the leaves, are *zhinst-toolkit* `Parameters`. In case you wonder what a certain parameter does, its object representation offers an insightful description.

In [5]:
hdawg.nodetree.system.clocks.referenceclock.source(0)

0

The parameters are callable to set and get the instrument values.

In [6]:
maxvolt=1.0
hdawg.nodetree.sigouts[0].range(maxvolt)
hdawg.nodetree.sigouts[1].range(maxvolt)

1.0

The instrument driver's *nodetree* property is created whenever it connects the device to the data server. This ensures that the available nodes and parameters in the nodetree are always up to date with the latest version of the device's firmware. 

Depending on the device, the instrument drivers also have properties other than the `nodetree`. Those are called *Modules* and depend on the type of device at hand and the options installed on it. Modules can for example be the *Data Acquisition Module* or the *Sweeper Module*. The devices with AWGs such as the HDAWG, UHFQA and the UHFLI (if option is installed) feature *AWG Modules*. Each *AWG Module* controls two chanels and features high-level functionality for programming sequences of waveforms and generating IQ-Modulation signals.

For more information, see the package documentation of the `zhinst-toolkit` and the following examples. 

In [7]:
hdawg.nodetree.system.awg.oscillatorcontrol(1)

1

In [8]:
hdawg.awgs[0].set_sequence_params(sequence_type="Custom",path="C:/Program Files/Zurich Instruments/LabOne/WebServer/awg/src/test.seqc",custom_params=[],)
hdawg.awgs[0].compile()

Compilation successful
hdawg1-0: Sequencer status: ELF file uploaded


In [9]:
hdawg.nodetree.triggers.out[0].source(4)
hdawg.nodetree.triggers.out[1].source(6)

6

In [10]:
iqfreq=100e6
offset1=0.01
offset2=0.02
hdawg.nodetree.oscs[0].freq(iqfreq)
hdawg.nodetree.oscs[1].freq(iqfreq)
hdawg.awgs[0].gain1=1.0
hdawg.awgs[0].gain2=0.95
hdawg.nodetree.sigouts[0].offset(offset1)
hdawg.nodetree.sigouts[1].offset(offset2)
hdawg.awgs[0].modulation_phase_shift=98
hdawg.awgs[0].enable_iq_modulation()

In [11]:
hdawg.awgs[0].run()

In [12]:
hdawg.awgs[0].output1("on")   
hdawg.awgs[0].output2("on")

'on'

In [13]:
hdawg.awgs[0].output1("off")   
hdawg.awgs[0].output2("off")

'off'

In [14]:
hdawg.awgs[0].disable_iq_modulation()

0